# Data Quality 

Dopo aver completato la fase di Data Ingestion tramite lo scraping il secondo step è quello di valutare la qualità dei dati raccolti.  
Il dataframe studiato presenta 18 colonne. Ogni colonna presenta delle differenze dalle altre e ci si aspetta determinati valori da queste.

Dopo aver settato:
1. la datasource
2. suite e le relative expectations (si rimanda al noteook SuiteSetUp_main)
3. creazione del checkpoint e validazione

Si condurrà una validazione del sample

In [2]:
import great_expectations as gx
from ruamel import yaml
from great_expectations.data_context.types.resource_identifiers import ExpectationSuiteIdentifier
import os
import sys
sys.path.insert(0, '../funzioni')
from funzioni import *

In [ ]:
context = gx.get_context()

### Configurazione della datasource 

In [ ]:
datasource_config: dict = {
    "name": "glassdoor_scraping",
    "class_name": "Datasource",
    "module_name": "great_expectations.datasource",
    "execution_engine": {
        "class_name": "PandasExecutionEngine",
        "module_name": "great_expectations.execution_engine",
    },
     "data_connectors": {
        "all": {
            "class_name": "InferredAssetFilesystemDataConnector",
            "base_directory": "../data",
            "default_regex": {"pattern": "(.*)\\.csv",
            "group_names": ["data_asset_name"]},
            #"batch_spec_passthrough": {
            #    "reader_method": "read_csv",
            #    "reader_options": {
            #        "header": True,
            #        "inferSchema": True,
            #    },
            #},
        }
    },
}

In [ ]:
#check configurazione
context.test_yaml_config(yaml.dump(datasource_config))

In [ ]:
## try to add
try:
    context.get_datasource(datasource_config["name"])
except ValueError:
    context.add_datasource(**datasource_config)
else:
    print(
        f"The datasource {datasource_config['name']} already exists in your Data Context!"
    )

### Indicazione delle suite

Si rimanda al noteook SuiteSetUp_main per eventuali modifiche della stessa

In [ ]:
a = context.list_expectation_suite_names()
suite_identifier = ExpectationSuiteIdentifier(expectation_suite_name=a[0])
context.build_data_docs(resource_identifiers=[suite_identifier])
context.open_data_docs(resource_identifier=suite_identifier)
print('http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/expectations/'+ a[0] +'.html')

### Creazione del checkpoint e validazione

E' stata creata una funzione che prendendo come input il data-asset (sotto gruppo della data-source) crerà per ogni data-asset un checkpoint.
Poiché ogni data-asset corrisponde al giorno in cui è stato lanciato il processo di data-ingestion verrà creato un checkpoint per ogni giorno in cui appunto ha girato il processo di data-ingestion

Per le funzioni si rimanda alla percorso `../funzioni/funzioni.py`.
Per ogni validazione del checkpoint saranno disponibili i risultati in formato HTML

In [3]:
run_dataq()

Questa la documentazione sulla suite: Main 
 http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/expectations/Main.html


Calculating Metrics:   0%|          | 0/240 [00:00<?, ?it/s]

Consulta i risultati del data_asset_name "scraping_all_20230131" al link:
http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/validations/Main/20230302-212348-scraping_all_20230131/20230302T212348.975967Z/a8c2603936f08f68234cfeb52d848e66.html


Calculating Metrics:   0%|          | 0/240 [00:00<?, ?it/s]

Consulta i risultati del data_asset_name "scraping_all_20230201" al link:
http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/validations/Main/20230302-212352-scraping_all_20230201/20230302T212352.759815Z/6846293c9834bd8f9f0498067de4cc79.html


...

Consulta i risultati del data_asset_name "scraping_all_20230228" al link:
http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/validations/Main/20230302-212513-scraping_all_20230228/20230302T212513.845939Z/fbf67c19836929edb6be46e22e84d420.html


Calculating Metrics:   0%|          | 0/240 [00:00<?, ?it/s]

Consulta i risultati del data_asset_name "scraping_all_20230301" al link:
http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/validations/Main/20230302-212517-scraping_all_20230301/20230302T212517.761606Z/29ab79bcc15897ac1099d480a1c3cdc1.html


Calculating Metrics:   0%|          | 0/240 [00:00<?, ?it/s]

Consulta i risultati del data_asset_name "scraping_all_20230302" al link:
http://localhost:9000/view/great_expectations/uncommitted/data_docs/local_site/validations/Main/20230302-212521-scraping_all_20230302/20230302T212521.767271Z/7887d0883a8a229b88c80fba8e4f2cd7.html
Checkpoint aggiornati o aggiunti:
 ['checkpoint_20230131', 'checkpoint_20230201', 'checkpoint_20230202', 'checkpoint_20230203', 'checkpoint_20230204', 'checkpoint_20230205', 'checkpoint_20230206', 'checkpoint_20230207', 'checkpoint_20230208', 'checkpoint_20230209', 'checkpoint_20230210', 'checkpoint_20230211', 'checkpoint_20230212', 'checkpoint_20230213', 'checkpoint_20230214', 'checkpoint_20230215', 'checkpoint_20230216', 'checkpoint_20230217', 'checkpoint_20230218', 'checkpoint_20230219', 'checkpoint_20230220', 'checkpoint_20230221', 'checkpoint_20230222', 'checkpoint_20230224', 'checkpoint_20230225', 'checkpoint_20230226', 'checkpoint_20230227', 'checkpoint_20230228', 'checkpoint_20230301', 'checkpoint_20230302', 'mai